In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp conjugates

# Conjugates

> This module contains scripts to pre-calculate the variational Bayes step, which is the most computationally demanding step in DGLM updating and forecasting. These functions are called automatically and do not need to be directly referenced by a user.

In [ ]:
#exporti
import numpy as np

from scipy.special import digamma
from scipy import optimize as opt
from functools import partial

from pybats.shared import trigamma, load_interpolators, load_sales_example

import pickle
import zlib
import os

ModuleNotFoundError: No module named 'pybats'

In [ ]:
#export
def beta_approx(x, ft, qt):
    x = x ** 2
    return np.array([digamma(x[0]) - digamma(x[1]) - ft,
                     trigamma(x=x[0]) + trigamma(x=x[1]) - qt]).reshape(-1)

In [ ]:
#export
def gamma_approx(x, ft, qt):
    x = x ** 2
    return np.array([digamma(x[0]) - np.log(x[1]) - ft, trigamma(x=x[0]) - qt]).reshape(-1)

In [ ]:
#export
def gamma_alpha_approx(x, qt):
    x = x**2
    return np.array([trigamma(x=x[0]) - qt]).reshape(-1)

In [ ]:
#export
def pois_alpha_param(qt, alpha=1.):
    sol = opt.root(partial(gamma_alpha_approx, qt=qt), x0=np.sqrt(np.array([alpha])), method='lm')
    return sol.x ** 2

In [ ]:
#export
def gamma_solver(ft, qt, alpha=1., beta=1.):

    # If q_t is is small, can use an approximation
    if qt < 0.0001:
        alpha = 1/qt
        beta = np.exp(digamma(alpha) - ft)
        return np.array([alpha, beta])

    # all else fails, do the optimization for alpha, followed by an exact soln for beta
    alpha = pois_alpha_param(qt)[0]
    beta = np.exp(digamma(alpha) - ft)
    return np.array([alpha, beta])

In [ ]:
#export
def beta_solver(ft, qt, alpha=1., beta=1.):

    # If qt is small, likely consistent with a large alpha, beta - can use an approximation
    # Ref: West & Harrison, pg. 530
    alpha = (1 / qt) * (1 + np.exp(ft))
    beta = (1 / qt) * (1 + np.exp(-ft))
    if qt < 0.0025:
        return np.array([alpha, beta])


    # all else fails, do the optimization
    sol = opt.root(partial(beta_approx, ft=ft, qt=qt), x0=np.sqrt(np.array([alpha, beta])), method='lm')
    return sol.x ** 2

In [ ]:
#export
# generic conj function
def conj_params(ft, qt, alpha=1., beta=1., interp=False, solver_fn=None, interp_fn=None):
    # the shape of these can vary a lot, so standardizing here.
    ft, qt = np.ravel(ft)[0], np.ravel(qt)[0]

    # do we want to interpolate?
    if interp and interp_fn is not None:
        # we may be asking for a value that's outside the interp range
        if interp_fn.ft_lb < ft < interp_fn.ft_ub and \
                interp_fn.qt_lb**2 < qt < interp_fn.qt_ub**2:
            return interp_fn(ft, qt)
    # all else fails, do the optimization
    return solver_fn(ft, qt, alpha, beta)

In [ ]:
#exporti
interp_beta, interp_gamma = load_interpolators()
assert interp_beta is not None
assert interp_gamma is not None

NameError: name 'load_interpolators' is not defined

In [ ]:
#export
# specific conjugate params functions
bern_conjugate_params = partial(conj_params, solver_fn=beta_solver, interp_fn=interp_beta, interp=True)
pois_conjugate_params = partial(conj_params, solver_fn=gamma_solver, interp_fn=interp_gamma, interp=True)
bin_conjugate_params = partial(conj_params, solver_fn=beta_solver, interp_fn=interp_beta, interp=True)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_dglm.ipynb.
Converted 01_update.ipynb.
Converted 02_forecast.ipynb.
Converted 03_define_models.ipynb.
Converted 04_seasonal.ipynb.
Converted 05_analysis.ipynb.
Converted 06_conjugates.ipynb.
Converted 07_point_forecast.ipynb.
Converted 08_loss_functions.ipynb.
Converted 09_plot.ipynb.
Converted 10_shared.ipynb.
Converted 11_dcmm.ipynb.
Converted 12_dbcm.ipynb.
Converted 13_latent_factor.ipynb.
Converted 14_latent_factor_fxns.ipynb.
Converted index.ipynb.
